## Data Science - Group 15

### Topic: Toronto E-Bike demographic, travel and public opinion data

In [1]:
##### Data Ingestion

import pandas as pd

raw_data_path = "../data/E-Bike_Survey_Responses.csv"
survey = pd.read_csv(raw_data_path)
print(survey.head())


          Timestamp 1. What age range do you fall in?   Sex  \
0  2013-04-10 12:10                          35 to 49  Male   
1  2013-04-10 12:30                          18 to 34  Male   
2  2013-04-10 12:33                         50 to 64   Male   
3  2013-04-10 12:52                         50 to 64   Male   
4  2013-04-10 13:24                          18 to 34  Male   

  How would you describe your level of physical health?  \
0                                               Good      
1                                          Excellent      
2                                               Good      
3                                               Good      
4                                          Very good      

  What level of education have you reached? What is your household income?  \
0                             Post graduate                         $100K+   
1                         University degree                   $40K to $59K   
2                         Univer